## Stream features from Web events using Signals

This notebook creates a new feature view using the SDK, tests it on the atomic events table and applies.

### Flow of data

```mermaid
flowchart LR
    sp(Snowplow Pipeline)
    stream[/Stream processing/]
    signals(Signals)

    sp --> stream
    stream --> signals
```

---

In [ ]:
from snowplow_signals import Signals

sp_signals = Signals(api_url="http://localhost:8000")

### Define a new feature

This block creates a single feature definition including the logic how it should be calculated (it's filters and aggregation).

The feature calculates the number of add to cart ecommerce events.

In [ ]:
from snowplow_signals import (
    Attribute,
    Criteria,
    Criterion,
    Event,
)

count_page_pings = Attribute(
    name="count_page_pings",
    type="int32",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_ping",
            version="1-0-0",
        )
    ],
    aggregation="counter",
)

visited_pages = Attribute(
    name="visited_pages",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="page_view",
            version="1-0-0",
        )
    ],
    aggregation="unique_list",
    property="page_title",
)

clicked_links_on_homepage = Attribute(
    name="clicked_links",
    type="string_list",
    events=[
        Event(
            vendor="com.snowplowanalytics.snowplow",
            name="link_click",
            version="1-0-1",
        )
    ],
    aggregation="unique_list",
    property="unstruct_event_com_snowplowanalytics_snowplow_link_click_1:targetUrl",
    criteria=Criteria(
        all=[
            Criterion(
                property="page_urlpath",
                operator="=",
                value="/",
            ),
        ],
    ),
)

### Wrapping the feature in a feature view

All features need to be included in feature views that can be considered as "tables" of features.

Feature views are immutable and versioned.

In [ ]:
from snowplow_signals import View, session_entity

view = View(
    name="my_web_attributes",
    version=1,
    entity=session_entity,
    attributes=[
        count_page_pings,
        visited_pages,
        clicked_links_on_homepage,
    ],
)

### Testing the feature view

Execute the feature view on the last one hour of data from the atomic events table to verify that it works correctly.

In [ ]:
data = sp_signals.test(
    view=view,
    app_ids=["website"],
)
data

### Applying the feature view to Signals

The following block pushes the feature view definition to the Signals API and makes it available for processing.

In [ ]:
applied = sp_signals.apply([view])
print(f"{len(applied)} objects applied")